## Build ScannerAgent and Messennger Agent

In [10]:
import os 
from dotenv import load_dotenv
import logging 
import requests
from openai import OpenAI

### Internal Classes 
from agents.deals import ScrapedDeal, DealSelection

In [2]:
load_dotenv()
MODEL = "gpt-5-mini"
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████| 3/3 [01:24<00:00, 28.16s/it]


In [4]:
len(deals)

30

In [5]:
print(deals[10].describe())

Title: Samsung Winter Tablet Deals at Best Buy from $160 + free shipping

Details: Save on 20 models, with up to $200 off. We've pictured the Samsung Galaxy Tab S10 Ultra 256GB 14.6" WiFi Tablet for $999.99 ($49 low). My Best Buy members get free shipping. (It's free to join. Shipping is free for everyone over $35. Pickup may also be available.)   Buy Now at Best Buy

Features: 

URL: https://www.dealnews.com/Samsung-Winter-Tablet-Deals-at-Best-Buy-from-160-free-shipping/21802169.html?iref=rss-c39


## GPT-5-Mini Parsing -- to summarize deals and identify their price into structured outputs

In [6]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [7]:
def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += "\n\n".join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX

    return user_prompt

In [8]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": make_user_prompt(deals)}
]

In [11]:
response = openai.chat.completions.parse(
    model=MODEL, 
    messages=messages, 
    response_format=DealSelection,
    reasoning_effort="minimal"
)

In [19]:
results = response.choices[0].message.parsed

print(results.deals)

[Deal(product_description='Apple AirPods Max are over-ear wireless headphones with active noise cancellation, spatial audio, and computational audio powered by Apple silicon. They feature premium build materials, comfortable padded ear cushions, and on-head detection with automatic device switching across Apple devices. The headphones support high-fidelity audio, transparency mode, and onboard controls for playback and volume.', price=429.0, url='https://www.dealnews.com/Best-Buy-Apple-Air-Pods-Max-Winter-Deals-100-off-free-shipping/21802177.html?iref=rss-c142'), Deal(product_description='LG C5 Series OLED65C5PUA is a 65-inch 4K OLED TV with self-emissive pixels for deep blacks and wide viewing angles, a 120 Hz refresh rate for smoother motion, and support for HLG, HDR10, and Dolby Vision. It integrates Alexa voice control and smart TV features for streaming and apps, delivering cinematic picture quality and responsive performance suitable for gaming and movie watching.', price=1400.0,

In [20]:
for deal in results.deals: 
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print("====================================================================")
    print("====================================================================")

Apple AirPods Max are over-ear wireless headphones with active noise cancellation, spatial audio, and computational audio powered by Apple silicon. They feature premium build materials, comfortable padded ear cushions, and on-head detection with automatic device switching across Apple devices. The headphones support high-fidelity audio, transparency mode, and onboard controls for playback and volume.
429.0
https://www.dealnews.com/Best-Buy-Apple-Air-Pods-Max-Winter-Deals-100-off-free-shipping/21802177.html?iref=rss-c142
LG C5 Series OLED65C5PUA is a 65-inch 4K OLED TV with self-emissive pixels for deep blacks and wide viewing angles, a 120 Hz refresh rate for smoother motion, and support for HLG, HDR10, and Dolby Vision. It integrates Alexa voice control and smart TV features for streaming and apps, delivering cinematic picture quality and responsive performance suitable for gaming and movie watching.
1400.0
https://www.dealnews.com/products/LG/LG-C5-Series-OLED65-C5-PUA-65-4-K-HDR-OLE

In [21]:
root = logging.getLogger()
root.setLevel(logging.INFO)